In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from matplotlib.colors import PowerNorm, LogNorm
from datetime import datetime
from netCDF4 import Dataset, num2date
from datetime import datetime
import imageio


variable_name = ''
long_name = ''
file_name = ''